# Multiple-Choice Question Answering

In [1]:
!nvidia-smi
!lscpu

Sat Aug 14 16:12:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0
!pip install editdistance==0.5.3

     |████████████████████████████████| 316 kB 8.1 MB/s 
     |████████████████████████████████| 233 kB 59.6 MB/s 
     |████████████████████████████████| 1.4 MB 61.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=ba18ab44e326cdf6b8e50aed4afe89a918298ac667d96b2dc7338c0c0202fa53
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154609 sha256=8bf1582825770b347bb4b9d62c7d8219b2aa83e20568a68fe09a5a8d1dfd17f6
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import collections
import numpy as np
import pandas as pd
import editdistance

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForMultipleChoice
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer
from transformers.data.metrics.squad_metrics import compute_exact, compute_f1

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class MultipleChoiceQADataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Multiple Choice Question Answering. """

    def __init__(self, questions, candidates, choices, answers, tokenizer, max_length, model_type):
        self.questions = questions
        self.candidates = candidates
        self.choices = choices
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.model_type = model_type

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, item):
        if self.model_type == "mt5":
            input_text = self.questions[item] + ' <sep> ' + ' <sep> '.join(self.candidates[item])
            encoding = self.tokenizer(
                input_text,
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
                padding='max_length',
                return_tensors="pt"
            )
            inputs = {
                'item': str(item),
                'question': self.questions[item],
                'candidates': ' <sep> '.join(self.candidates[item]),
                'input_text': input_text,
                'choice': self.choices[item],
                'answer': self.answers[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten()
            }
            return inputs
        else:
            choices_input_ids, choices_attention_masks, choices_token_type_ids = [], [], []
            for c in self.candidates[item]:
                text_a = ""  # empty context
                text_b = self.questions[item] + " " + c
                inputs = self.tokenizer(
                    text_a,
                    text_b,
                    add_special_tokens=True,
                    max_length=self.max_length,
                    padding="max_length",
                    truncation=True,
                    return_overflowing_tokens=True
                )
                choices_input_ids.append(inputs.input_ids[0])
                choices_attention_masks.append(inputs.attention_mask[0])
                choices_token_type_ids.append(inputs.token_type_ids[0])

            inputs = {
                'item': str(item),
                'question': self.questions[item],
                'candidates': ' <sep> '.join(self.candidates[item]),
                'choice': int(self.choices[item]) - 1,
                'answer': self.answers[item],
                'input_ids': torch.LongTensor(choices_input_ids),
                'attention_mask': torch.LongTensor(choices_attention_masks),
                'token_type_ids': torch.LongTensor(choices_token_type_ids)
            }
            return inputs


class MultipleChoiceQA:
    def __init__(self, model_name, model_type):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type.lower() == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        elif model_type.lower() in ["mbert", "parsbert", "wikibert"]:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.model = AutoModelForMultipleChoice.from_pretrained(self.model_name, config=self.config)
            self.model_type = model_type.lower()
        else:
            print(f'model_type not supported!')
            return

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() in ["parsinlu", "parsinlu-literature", "parsinlu-math_and_logic",
                                    "parsinlu-common_knowledge"]:
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            questions, candidates, choices, answers = [], [], [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    json_line = json.loads(line.strip())
                    question = json_line['question']
                    candidate_answers = json_line['candidates']
                    choice = json_line['answer']
                    answer = candidate_answers[int(json_line['answer']) - 1]

                    questions.append(question)
                    candidates.append(candidate_answers)
                    choices.append(choice)
                    answers.append(answer)
            return questions, candidates, choices, answers

    def multiple_choice_qa_inference(self, questions, candidates, device, max_length=512):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        input_ids, attention_masks, token_type_ids = [], [], []
        for q, cs in zip(questions, candidates):
            choices_input_ids, choices_attention_masks, choices_token_type_ids = [], [], []
            for c in cs:
                text_a = ""  # empty context
                text_b = q + " " + c
                inputs = self.tokenizer(
                    text_a,
                    text_b,
                    add_special_tokens=True,
                    max_length=max_length,
                    padding="max_length",
                    truncation=True,
                    return_overflowing_tokens=True,
                )
                choices_input_ids.append(inputs.input_ids[0])
                choices_attention_masks.append(inputs.attention_mask[0])
                choices_token_type_ids.append(inputs.token_type_ids[0])
            input_ids.append(choices_input_ids)
            attention_masks.append(choices_attention_masks)
            token_type_ids.append(choices_token_type_ids)

        input_ids = torch.LongTensor(input_ids).to(device)
        attention_masks = torch.LongTensor(attention_masks).to(device)
        token_type_ids = torch.LongTensor(token_type_ids).to(device)

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        outputs = self.model(input_ids=input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
        predictions = torch.argmax(outputs.logits, dim=1)
        return [(questions[i], candidates[i], candidates[i][p.item()]) for i, p in enumerate(predictions)]

    def mt5_multiple_choice_qa_inference(self, questions, candidates, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        new_input = []
        for q, cs in zip(questions, candidates):
            new_input.append(q + ' <sep> ' + ' <sep> '.join(cs))

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)

        outputs = self.model.generate(input_ids=input_ids, attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return [(questions[i], candidates[i], p) for i, p in enumerate(predictions)]

    def evaluation(self, questions, candidates, choices, answers, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(questions) != len(candidates):
            print('length of two inputs is not equal!!')
            return
        if len(choices) != len(answers):
            print('length of choices and answers is not equal!!')
            return
        if len(questions) != len(answers):
            print('length of inputs and answers is not equal!!')
            return

        dataset = MultipleChoiceQADataset(questions=questions, candidates=candidates, choices=choices, answers=answers,
                                          tokenizer=self.tokenizer, max_length=max_length, model_type=self.model_type)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#question:{len(questions)}, #candidates:{len(candidates)}, #answer:{len(answers)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_choices, predicted_choices = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_choices = batch['choice']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_choices = b_choices.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_choices)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            golden_choices.extend(b_choices.cpu().detach().numpy().tolist())
            b_predictions = torch.argmax(b_outputs.logits, dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            predicted_choices.extend(b_predictions)

            for i in range(len(b_input_ids)):
                output_predictions.append((
                    batch['question'][i],
                    batch['candidates'][i].split(' <sep> '),
                    batch['choice'][i].item(),
                    batch['answer'][i],
                    b_predictions[i],
                    batch['candidates'][i].split(' <sep> ')[b_predictions[i]]
                ))

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(questions))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_choices, predicted_choices)))
        print("Test Precision: {}".format(precision_score(golden_choices, predicted_choices, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_choices, predicted_choices, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_choices, predicted_choices, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_choices, predicted_choices, digits=10)))
        return output_predictions

    def mt5_evaluation(self, questions, candidates, choices, answers, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(questions) != len(candidates):
            print('length of two inputs is not equal!!')
            return
        if len(choices) != len(answers):
            print('length of choices and answers is not equal!!')
            return
        if len(questions) != len(answers):
            print('length of inputs and answers is not equal!!')
            return

        dataset = MultipleChoiceQADataset(questions=questions, candidates=candidates, choices=choices, answers=answers,
                                          tokenizer=self.tokenizer, max_length=max_length, model_type="mt5")
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#question:{len(questions)}, #candidates:{len(candidates)}, #answer:{len(answers)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_time = 0
        output_predictions = []
        golden_choices, predicted_choices, exact_score_list, f1_score_list = [], [], [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)

            for i in range(len(b_input_ids)):
                if b_predictions[i] in batch['candidates'][i].split(' <sep> '):
                    predicted_choice = str(batch['candidates'][i].split(' <sep> ').index(b_predictions[i]) + 1)
                else:
                    normalized_edit_distance_list = [
                        editdistance.distance(ca, b_predictions[i]) / max(len(ca), len(b_predictions[i])) for ca in
                        batch['candidates'][i].split(' <sep> ')
                    ]
                    predicted_choice = str(normalized_edit_distance_list.index(min(normalized_edit_distance_list)) + 1)

                golden_choices.append(batch['choice'][i])
                predicted_choices.append(predicted_choice)

                exact_score_list.append(compute_exact(batch['answer'][i], b_predictions[i]))
                f1_score_list.append(compute_f1(batch['answer'][i], b_predictions[i]))

                output_predictions.append((
                    batch['question'][i],
                    batch['candidates'][i].split(' <sep> '),
                    batch['choice'][i],
                    batch['answer'][i],
                    predicted_choice,
                    b_predictions[i],
                    exact_score_list[-1],
                    f1_score_list[-1]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(questions))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_choices, predicted_choices)))
        print("Test Precision: {}".format(precision_score(golden_choices, predicted_choices, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_choices, predicted_choices, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_choices, predicted_choices, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_choices, predicted_choices, digits=10)))

        total = len(exact_score_list)
        evaluation_results = collections.OrderedDict(
            [
                ("exact", 100.0 * sum(exact_score_list) / total),
                ("f1", 100.0 * sum(f1_score_list) / total),
                ("total", total),
            ]
        )
        print("evaluation results:\n", evaluation_results)

        return output_predictions


In [6]:
model_name='persiannlp/mbert-base-parsinlu-multiple-choice'
mcqa_model = MultipleChoiceQA(model_name=model_name, model_type="mbert")
print(mcqa_model.config)

Downloading:   0%|          | 0.00/902 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/670M [00:00<?, ?B/s]

BertConfig {
  "_name_or_path": "persiannlp/mbert-base-parsinlu-multiple-choice",
  "architectures": [
    "BertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": "multiple_choice_all",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": 

## Sample Inference

In [7]:
question_list = [
    "وسیع ترین کشور جهان کدام است؟",
    "طامع یعنی ؟",
    "زمینی به ۳۱ قطعه متساوی مفروض شده است و هر روز مساحت آماده شده برای احداث، دو برابر مساحت روز قبل است.اگر پس از (۵ روز) تمام زمین آماده شده باشد، در چه روزی یک قطعه زمین آماده شده"
]
candidate_list=[
    ["آمریکا", "کانادا", "روسیه", "چین"],
    ["آزمند", "خوش شانس", "محتاج", "مطمئن"],
    ["روز اول", "روز دوم", "روز سوم", "هیچکدام"]
]
mcqa_model.multiple_choice_qa_inference(question_list, candidate_list, device)

[('وسیع ترین کشور جهان کدام است؟',
  ['آمریکا', 'کانادا', 'روسیه', 'چین'],
  'آمریکا'),
 ('طامع یعنی ؟', ['آزمند', 'خوش شانس', 'محتاج', 'مطمئن'], 'آزمند'),
 ('زمینی به ۳۱ قطعه متساوی مفروض شده است و هر روز مساحت آماده شده برای احداث، دو برابر مساحت روز قبل است.اگر پس از (۵ روز) تمام زمین آماده شده باشد، در چه روزی یک قطعه زمین آماده شده',
  ['روز اول', 'روز دوم', 'روز سوم', 'هیچکدام'],
  'روز اول')]

## Multiple-Choice Dataset


In [8]:
!git clone https://github.com/persiannlp/parsinlu
!ls parsinlu
!ls parsinlu/data/multiple-choice/

Cloning into 'parsinlu'...
remote: Enumerating objects: 1434, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1434 (delta 110), reused 139 (delta 82), pack-reused 1252
Receiving objects: 100% (1434/1434), 27.81 MiB | 19.53 MiB/s, done.
Resolving deltas: 100% (913/913), done.
data  LICENSE  README.md  requirements.txt  scripts  src
test_ck.jsonl  test_lit.jsonl  train.jsonl
test.jsonl     test_ml.jsonl   valid.jsonl


### Samples with literature as their category

In [9]:
test_questions_lit, test_candidates_lit, test_choices_lit, test_answers_lit = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu-literature", dataset_file="./parsinlu/data/multiple-choice/test_lit.jsonl")
print(test_questions_lit[0])
print(test_candidates_lit[0])
print(test_choices_lit[0])
print(test_answers_lit[0])
print(len(test_questions_lit))
print(len(test_candidates_lit))
print(len(test_choices_lit))
print(len(test_answers_lit))

رابطه‌ی شیر با جنگل مثل رابطه‌ی
['سرباز است با پادگان', 'اتوبوس است با ایستگاه', 'هواپیما است با آسمان', 'کشتی است با بندر']
3
هواپیما است با آسمان
350
350
350
350


In [10]:
!nvidia-smi
!lscpu

Sat Aug 14 16:14:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    28W /  70W |   8736MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
evaluation_output = mcqa_model.evaluation(test_questions_lit, test_candidates_lit, test_choices_lit, test_answers_lit, device, max_length=mcqa_model.config.max_position_embeddings, batch_size=64)

#question:350, #candidates:350, #answer:350
#batch: 6
Start to evaluate test data ...
inference time for step 0: 0.04121049400009724
inference time for step 1: 0.00961905300005128
inference time for step 2: 0.009158184999932928
inference time for step 3: 0.009137297000052058
inference time for step 4: 0.009354520999977467
inference time for step 5: 0.009352933999934976
average loss: 1.386294960975647
total inference time: 0.08783248400004595
total inference time / #samples: 0.0002509499542858456
Test Accuracy: 0.21428571428571427
Test Precision: 0.04591836734693877
Test Recall: 0.21428571428571427
Test F1-Score(weighted average): 0.07563025210084034
Test classification Report:
              precision    recall  f1-score   support

           0  0.2142857143 1.0000000000 0.3529411765        75
           1  0.0000000000 0.0000000000 0.0000000000       102
           2  0.0000000000 0.0000000000 0.0000000000        93
           3  0.0000000000 0.0000000000 0.0000000000        80

    ac

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))

رابطه‌ی شیر با جنگل مثل رابطه‌ی	['سرباز است با پادگان', 'اتوبوس است با ایستگاه', 'هواپیما است با آسمان', 'کشتی است با بندر']	2	هواپیما است با آسمان	0	سرباز است با پادگان
رابطه ي بخار با یخ مثل رابطه ي:	['خمیر است با نان', 'گندم است با آرد', 'غوره است با کشمش', 'باران است با برف']	2	غوره است با کشمش	0	خمیر است با نان
در عبارت زیر، به‌ترتیب « مضاف‌الیه مضاف‌الیه، صفت مضاف‌الیه و متمم اسم» کدام است؟
«مطالعه تفاسیر قرآن، روح اشعار حافظ شیراز را جلایی خاص بخشیده و از غزلیات این شاعر بی‌بدیل می‌توان به مهارت
	['قرآن، این، کشف', 'حافظ، عرفانی، غزلیات', 'شیراز، این شاعر، مهارت', 'رموز، بی\u200cبدیل، کشف رموز عرفانی']	0	قرآن، این، کشف	0	قرآن، این، کشف
مشهورترین شاعر رمانتیک قرن نوزدهم فرانسه چه کسی است؟	['ولتر', 'ویکتورهوگو', 'لافونتن', 'ژان ژاک  روسو']	1	ویکتورهوگو	0	ولتر
مفرد كدام كلمه صحيح است	['الوان : لون', 'حواس : احساس', 'اعضا: عضوها', 'الف وب']	2	اعضا: عضوها	0	الوان : لون
کدام عبارت، نادرست است؟	['شاعر منظومه\u200cهای حماسی مصنوع، با داستان\u200cهای پهلوانی مدوّن و معینی سر و کار دارد.'

In [13]:
output_file_name = "multiple_choice_qa_literature_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Samples with math_and_logic as their category

In [14]:
test_questions_ml, test_candidates_ml, test_choices_ml, test_answers_ml = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu-math_and_logic", dataset_file="./parsinlu/data/multiple-choice/test_ml.jsonl")
print(test_questions_ml[0])
print(test_candidates_ml[0])
print(test_choices_ml[0])
print(test_answers_ml[0])
print(len(test_questions_ml))
print(len(test_candidates_ml))
print(len(test_choices_ml))
print(len(test_answers_ml))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟
['2A', '2A+B', '3A+B', 'A-B']
2
2A+B
350
350
350
350


In [15]:
!nvidia-smi
!lscpu

Sat Aug 14 16:15:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    35W /  70W |  10608MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
evaluation_output = mcqa_model.evaluation(test_questions_ml, test_candidates_ml, test_choices_ml, test_answers_ml, device, max_length=mcqa_model.config.max_position_embeddings, batch_size=64)

#question:350, #candidates:350, #answer:350
#batch: 6
Start to evaluate test data ...
inference time for step 0: 0.20227467200004412
inference time for step 1: 0.008978460999969684
inference time for step 2: 0.01152231900005063
inference time for step 3: 0.009117444000025898
inference time for step 4: 0.011428023999997095
inference time for step 5: 0.009102096999981768
average loss: 1.386294960975647
total inference time: 0.2524230170000692
total inference time / #samples: 0.0007212086200001976
Test Accuracy: 0.33714285714285713
Test Precision: 0.11366530612244898
Test Recall: 0.33714285714285713
Test F1-Score(weighted average): 0.17001221001221
Test classification Report:
              precision    recall  f1-score   support

           0  0.3371428571 1.0000000000 0.5042735043       118
           1  0.0000000000 0.0000000000 0.0000000000        99
           2  0.0000000000 0.0000000000 0.0000000000        81
           3  0.0000000000 0.0000000000 0.0000000000        52

    accura

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟	['2A', '2A+B', '3A+B', 'A-B']	1	2A+B	0	2A
در ادامه این رشته چه عددی باید نوشت؟ ۹۱،۸۶،۷۶،۶۱،...	['۴۶', '۴۱', '۵۱', '۳۶']	1	۴۱	0	۴۶
50 تا 20 تا برابر است با ......	['10000', '100', '1000', '500']	2	1000	0	10000
در ادامه این رشته چه عددی باید نوشت؟             3, 5, 5, 9, 7, 13, 9, …	['17', '11', '14', '15']	0	17	0	17
مساحت مربع ۸ ،p برابر مساحت مربع Q است. نسبت قطر مربع p به ضلع مربع Q کدامست؟	['۴', '۲', '۳', '۱']	0	۴	0	۴
%50 عدد 24 برابر است با ....	['4', '6', '10', '12']	3	12	0	4
کدام عدد نزدیکتر۷ به است؟	['۴', '۶', '۹', '۱۱']	1	۶	0	۴
چند درصد ۵۰۰ برابر ۵۰ می‌شود؟	['۱', '۱۰', '۲۰', '۳۰']	1	۱۰	0	۱
قیمت یک کالا %۲۵ تخفیف داده شده است برای آنکه این کالا به قیمت قبل از تخفیف فروخته شود چند درصد باید به قیمت آن افزوده گردد؟	['۲۵', '۲۰', '۳۳.۳۳', 'هیچکدام']	2	۳۳.۳۳	0	۲۵
حاصل عبارت ۵ - ۳ برابر است با ؟	['-2', '2', '1', '-1']	0	-2	0	-2
حاصل عبارت ۴۴ + ۲۳ برابر است با ؟	['68', '6

In [18]:
output_file_name = "multiple_choice_qa_math_and_logic_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Samples with common_knowledge as their category

In [19]:
test_questions_ck, test_candidates_ck, test_choices_ck, test_answers_ck = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu-common_knowledge", dataset_file="./parsinlu/data/multiple-choice/test_ck.jsonl")
print(test_questions_ck[0])
print(test_candidates_ck[0])
print(test_choices_ck[0])
print(test_answers_ck[0])
print(len(test_questions_ck))
print(len(test_candidates_ck))
print(len(test_choices_ck))
print(len(test_answers_ck))

کدام کشور اولین تولید کننده خرما در جهان است؟
['ایران', 'عربستان', 'عراق', 'سوریه']
1
ایران
350
350
350
350


In [20]:
!nvidia-smi
!lscpu

Sat Aug 14 16:16:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   79C    P0    46W /  70W |  10608MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
evaluation_output = mcqa_model.evaluation(test_questions_ck, test_candidates_ck, test_choices_ck, test_answers_ck, device, max_length=mcqa_model.config.max_position_embeddings, batch_size=64)

#question:350, #candidates:350, #answer:350
#batch: 6
Start to evaluate test data ...
inference time for step 0: 0.17507602299997416
inference time for step 1: 0.00920920400005798
inference time for step 2: 0.009225366999999096
inference time for step 3: 0.009466632999988178
inference time for step 4: 0.00888994999991155
inference time for step 5: 0.008806349999986196
average loss: 1.386294960975647
total inference time: 0.22067352699991716
total inference time / #samples: 0.0006304957914283348
Test Accuracy: 0.28
Test Precision: 0.07862464183381089
Test Recall: 0.28
Test F1-Score(weighted average): 0.12277404921700225
Test classification Report:
              precision    recall  f1-score   support

           0  0.2808022923 1.0000000000 0.4384787472        98
           1  0.0000000000 0.0000000000 0.0000000000        88
           2  0.0000000000 0.0000000000 0.0000000000        94
           3  0.0000000000 0.0000000000 0.0000000000        70

    accuracy                      0.2

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))

کدام کشور اولین تولید کننده خرما در جهان است؟	['ایران', 'عربستان', 'عراق', 'سوریه']	0	ایران	0	ایران
مقام رهبری در قانون اساسی جمهوری اسلامی دارای چه کار ویژه ای است؟	['ریاست کشور', 'نظارت عالیه', 'تنظیم کننده قوای سه گانه', 'حاکمیت مطلق']	2	تنظیم کننده قوای سه گانه	0	ریاست کشور
الماس سخت تر است یا گرانیت؟	['الماس', 'گرانیت', '', '']	0	الماس	0	الماس
طبق قانون اساسی شورای نگهبان طی چند روز از تاریخ وصول باید نظر خود را نسبت به مصوبات مجلس
	['یک ماه', 'ده روز', 'دو هفته', 'تا حصول اطمینان']	1	ده روز	0	یک ماه
در کشور ایران بیشترین نرخ بیکاری متعلق به کدام یک از گزینه های زیر میباشد؟	['دارندگان مدرک تحصیلی تکمیلی', 'افراد زیر دیپلم', 'افراد بالای 50 سال', 'فارغ التحصیلان دانشگاهها']	0	دارندگان مدرک تحصیلی تکمیلی	0	دارندگان مدرک تحصیلی تکمیلی
ریاست اولین دوره مجلس شورای اسلامی بر عهده چه کسی بود؟	['آیت الله بهشتی', 'آیت الله کروبی', 'آیت الله رفسنجانی', 'آیت الله ناطق نوری']	3	آیت الله ناطق نوری	0	آیت الله بهشتی
سوره بیست و هفتم قرآن کریم کدامست؟	['سوره اسراء', 'سوره توبه', 'سوره واقعه', 'سو

In [23]:
output_file_name = "multiple_choice_qa_common_knowledge_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### All Samples

In [24]:
test_questions_all, test_candidates_all, test_choices_all, test_answers_all = mcqa_model.load_dataset_test_file(
    dataset_name="parsinlu", dataset_file="./parsinlu/data/multiple-choice/test.jsonl")
print(test_questions_all[0])
print(test_candidates_all[0])
print(test_choices_all[0])
print(test_answers_all[0])
print(len(test_questions_all))
print(len(test_candidates_all))
print(len(test_choices_all))
print(len(test_answers_all))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟
['2A', '2A+B', '3A+B', 'A-B']
2
2A+B
1050
1050
1050
1050


In [25]:
!nvidia-smi
!lscpu

Sat Aug 14 16:17:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    44W /  70W |  10608MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
evaluation_output = mcqa_model.evaluation(test_questions_all, test_candidates_all, test_choices_all, test_answers_all, device, max_length=mcqa_model.config.max_position_embeddings, batch_size=64)

#question:1050, #candidates:1050, #answer:1050
#batch: 17
Start to evaluate test data ...
inference time for step 0: 0.2715611620000118
inference time for step 1: 0.009469446000025528
inference time for step 2: 0.008967185000074096
inference time for step 3: 0.009212233999960517
inference time for step 4: 0.00958400799993342
inference time for step 5: 0.009290498999916963
inference time for step 6: 0.009146288000010827
inference time for step 7: 0.009280863000071804
inference time for step 8: 0.011202780000076018
inference time for step 9: 0.008858752999913122
inference time for step 10: 0.009483360999979595
inference time for step 11: 0.01733615299997382
inference time for step 12: 0.009144052999999985
inference time for step 13: 0.009205721000057565
inference time for step 14: 0.008829916000081539
inference time for step 15: 0.01344903099993644
inference time for step 16: 0.008766003000005185
average loss: 1.3862949539633358
total inference time: 0.4327874560000282
total inference ti

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output[:25]:
  print('{}\t{}\t{}\t{}\t{}\t{}'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))

تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟	['2A', '2A+B', '3A+B', 'A-B']	1	2A+B	0	2A
در ادامه این رشته چه عددی باید نوشت؟ ۹۱،۸۶،۷۶،۶۱،...	['۴۶', '۴۱', '۵۱', '۳۶']	1	۴۱	0	۴۶
50 تا 20 تا برابر است با ......	['10000', '100', '1000', '500']	2	1000	0	10000
در ادامه این رشته چه عددی باید نوشت؟             3, 5, 5, 9, 7, 13, 9, …	['17', '11', '14', '15']	0	17	0	17
مساحت مربع ۸ ،p برابر مساحت مربع Q است. نسبت قطر مربع p به ضلع مربع Q کدامست؟	['۴', '۲', '۳', '۱']	0	۴	0	۴
%50 عدد 24 برابر است با ....	['4', '6', '10', '12']	3	12	0	4
کدام عدد نزدیکتر۷ به است؟	['۴', '۶', '۹', '۱۱']	1	۶	0	۴
چند درصد ۵۰۰ برابر ۵۰ می‌شود؟	['۱', '۱۰', '۲۰', '۳۰']	1	۱۰	0	۱
قیمت یک کالا %۲۵ تخفیف داده شده است برای آنکه این کالا به قیمت قبل از تخفیف فروخته شود چند درصد باید به قیمت آن افزوده گردد؟	['۲۵', '۲۰', '۳۳.۳۳', 'هیچکدام']	2	۳۳.۳۳	0	۲۵
حاصل عبارت ۵ - ۳ برابر است با ؟	['-2', '2', '1', '-1']	0	-2	0	-2
حاصل عبارت ۴۴ + ۲۳ برابر است با ؟	['68', '6

In [28]:
output_file_name = "multiple_choice_qa_all_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for question, candidates, true_choice, true_answer, predicted_choice, predicted_answer in evaluation_output:
    output_file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(question, candidates, true_choice, true_answer, predicted_choice, predicted_answer))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()